# MODELO DE MACHINE LEARNING

Se ha utilizado RandomForestClassifier como modelo de machine learning. Esta elección se basa en la flexibilidad y buen rendimiento de los bosques aleatorios en problemas de clasificación.

Se han dividido los datos en conjuntos de entrenamiento y prueba para entrenar el modelo en un conjunto y evaluar su rendimiento en otro conjunto independiente.

Se han evaluado el modelo utilizando métricas comunes como la precisión, la recuperación (recall) y la puntuación F1-score, así como la matriz de confusión.

In [3]:
# Importar bibliotecas necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [4]:
# Cargar el conjunto de datos procesado
file_path_processed = "dataset_procesado.csv"
df_processed = pd.read_csv(file_path_processed)

# Dividir el conjunto de datos en características (X) y variable objetivo (y)
X = df_processed.drop("income", axis=1)
y = df_processed["income"]

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de Random Forest
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Mostrar resultados
print(f"Accuracy: {accuracy}")
print("\nClassification Report:\n", classification_rep)
print("\nConfusion Matrix:\n", conf_matrix)

Accuracy: 0.8532575291948371

Classification Report:
               precision    recall  f1-score   support

       <=50K       0.89      0.92      0.91      4988
        >50K       0.71      0.63      0.67      1520

    accuracy                           0.85      6508
   macro avg       0.80      0.78      0.79      6508
weighted avg       0.85      0.85      0.85      6508


Confusion Matrix:
 [[4597  391]
 [ 564  956]]


### MEJORANDO EL MODELO

In [5]:
from sklearn.model_selection import GridSearchCV

# Definir los parámetros a ajustar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Inicializar el modelo
model = RandomForestClassifier(random_state=42)

# Realizar búsqueda en cuadrícula
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros
print("Mejores Hiperparámetros:", grid_search.best_params_)

# Obtener el mejor modelo
best_model = grid_search.best_estimator_

Mejores Hiperparámetros: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}


Una búsqueda en cuadrícula para encontrar los mejores hiperparámetros para el modelo de Random Forest. Los hiperparámetros óptimos son:

max_depth: None
min_samples_leaf: 2
min_samples_split: 5
n_estimators: 200
Utilizar estos hiperparámetros para entrenar un nuevo modelo y evaluar su rendimiento.

In [6]:
# Crear y entrenar el modelo de Random Forest con los mejores hiperparámetros
best_model = RandomForestClassifier(
    random_state=42,
    max_depth=None,
    min_samples_leaf=2,
    min_samples_split=5,
    n_estimators=200
)

best_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_best = best_model.predict(X_test)

# Evaluar el rendimiento del nuevo modelo
accuracy_best = accuracy_score(y_test, y_pred_best)
classification_rep_best = classification_report(y_test, y_pred_best)
conf_matrix_best = confusion_matrix(y_test, y_pred_best)

# Mostrar resultados del nuevo modelo
print(f"Accuracy del mejor modelo: {accuracy_best}")
print("\nClassification Report del mejor modelo:\n", classification_rep_best)
print("\nConfusion Matrix del mejor modelo:\n", conf_matrix_best)

Accuracy del mejor modelo: 0.8660110633066994

Classification Report del mejor modelo:
               precision    recall  f1-score   support

       <=50K       0.89      0.94      0.91      4988
        >50K       0.76      0.63      0.69      1520

    accuracy                           0.87      6508
   macro avg       0.82      0.78      0.80      6508
weighted avg       0.86      0.87      0.86      6508


Confusion Matrix del mejor modelo:
 [[4682  306]
 [ 566  954]]


# AJUSTE DE HIPERPARAMETROS CON OPTUNA

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import optuna

# Cargar el conjunto de datos procesado
file_path_processed = "dataset_procesado.csv"
df_processed = pd.read_csv(file_path_processed)

# Dividir el conjunto de datos en características (X) y variable objetivo (y)
X = df_processed.drop("income", axis=1)
y = df_processed["income"]

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Función de objetivo para la optimización de Optuna
def objective(trial):
    # Definir espacio de búsqueda para hiperparámetros
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_categorical('max_depth', [None, 10, 20])
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 4)

    # Crear y entrenar el modelo de Random Forest con hiperparámetros sugeridos por Optuna
    model = RandomForestClassifier(
        random_state=42,
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )
    model.fit(X_train, y_train)

    # Realizar predicciones en el conjunto de prueba
    y_pred = model.predict(X_test)

    # Calcular la precisión del modelo
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# Crear un estudio Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)  # Puedes ajustar el número de ensayos (trials)

# Imprimir los mejores hiperparámetros y la precisión asociada
print("Mejores Hiperparámetros:", study.best_params)
print("Mejor Precisión:", study.best_value)

# Obtener el mejor modelo con los hiperparámetros optimizados
best_model = RandomForestClassifier(
    random_state=42,
    n_estimators=study.best_params['n_estimators'],
    max_depth=study.best_params['max_depth'],
    min_samples_split=study.best_params['min_samples_split'],
    min_samples_leaf=study.best_params['min_samples_leaf']
)

# Entrenar el mejor modelo
best_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_best = best_model.predict(X_test)

# Evaluar el rendimiento del mejor modelo
accuracy_best = accuracy_score(y_test, y_pred_best)
classification_rep_best = classification_report(y_test, y_pred_best)
conf_matrix_best = confusion_matrix(y_test, y_pred_best)

# Mostrar resultados del mejor modelo
print(f"Accuracy del mejor modelo: {accuracy_best}")
print("\nClassification Report del mejor modelo:\n", classification_rep_best)
print("\nConfusion Matrix del mejor modelo:\n", conf_matrix_best)

c:\Users\Terán\Documents\finalProject\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-01-23 20:05:21,868] A new study created in memory with name: no-name-0eda1451-cd73-4bcb-9b91-9b4719c6a874
[I 2024-01-23 20:05:28,453] Trial 0 finished with value: 0.8612476951444377 and parameters: {'n_estimators': 139, 'max_depth': 10, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.8612476951444377.
[I 2024-01-23 20:05:31,542] Trial 1 finished with value: 0.8606330669944684 and parameters: {'n_estimators': 66, 'max_depth': 10, 'min_samples_split': 9, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.8612476951444377.
[I 2024-01-23 20:05:43,518] Trial 2 finished with value: 0.8633988936693301 and parameters: {'n_estimators': 167, 'max_depth': 20, 'min_samples_split': 2, 'min_samples

Mejores Hiperparámetros: {'n_estimators': 180, 'max_depth': None, 'min_samples_split': 9, 'min_samples_leaf': 2}
Mejor Precisión: 0.8681622618315918
Accuracy del mejor modelo: 0.8681622618315918

Classification Report del mejor modelo:
               precision    recall  f1-score   support

       <=50K       0.89      0.94      0.92      4988
        >50K       0.76      0.63      0.69      1520

    accuracy                           0.87      6508
   macro avg       0.83      0.79      0.80      6508
weighted avg       0.86      0.87      0.86      6508


Confusion Matrix del mejor modelo:
 [[4694  294]
 [ 564  956]]
